In [1]:
%matplotlib inline
from time import sleep

import gym
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
env = gym.make('MountainCarContinuous-v0')

In [10]:
state = env.reset()
print(env.action_space)
for i in range(10):
    env.render()
    action = env.action_space.sample()
    state, reward, done, _ = env.step(action)
    print("{}: {} {} {} {}".format(i, action, state, reward, done))
    sleep(0.01)

Box(1,)
0: [ 0.58345008] [-0.58786448  0.00136423] -0.034041399137732846 False
1: [ 0.05778984] [-0.58593456  0.00192992] -0.0003339665550107158 False
2: [ 0.13608912] [-0.58333571  0.00259885] -0.0018520249177863546 False
3: [ 0.85119328] [-0.57901443  0.00432128] -0.07245299941040566 False
4: [-0.85792788] [-0.57556634  0.00344809] -0.07360402534656266 False
5: [-0.8257414] [-0.57296868  0.00259766] -0.06818488606597609 False
6: [-0.95956321] [-0.57144144  0.00152724] -0.09207615446189171 False
7: [ 0.66523969] [-0.56855876  0.00288268] -0.044254384660933656 False
8: [ 0.5563135] [-0.56450543  0.00405333] -0.03094847123959086 False
9: [ 0.7400243] [-0.55903603  0.0054694 ] -0.05476359594009044 False


In [11]:
learning_rate = 0.05

_inparam = 2
_h1 = 2

In [12]:
_input = tf.placeholder(shape=[1, _inparam], dtype=tf.float32)
w1 = tf.Variable(tf.random_uniform([_inparam, _h1],0 , 0.01))
b1 = tf.Variable(tf.random_uniform([1, _h1],0 , 0.01))
h1 = tf.tanh(tf.add(tf.matmul(_input, w1), b1))

w2 = tf.Variable(tf.random_uniform([_h1, 1],0 , 0.01))
b2 = tf.Variable(tf.random_uniform([1, 1],0 , 0.01))
Q = tf.tanh(tf.add(tf.matmul(h1, w2), b2))

Qe = tf.placeholder(dtype=tf.float32)
error = tf.reduce_sum(tf.square(Qe - Q))
trainer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(error)

In [13]:
beta = 0.9
init = tf.global_variables_initializer()

steps = [0]

with tf.Session() as sess:
    sess.run(init)
    for episode in range(10):
        state = env.reset()
        done = False
        print()
        print(episode)
        print()
        while not done:
            rand_val = np.random.rand(1)
            resQ = sess.run(Q, feed_dict={_input:[state]})
            if rand_val < beta:
                resQ *= -1
            next_state, reward, done, _ = env.step(resQ[0])
            if done and reward > 0:
                beta *= 0.8
            r = reward*-1
            
            next_Q = sess.run(Q, feed_dict={_input:[next_state]})
            resQ = r + 0.4*next_Q[0]
            sess.run([trainer], feed_dict={_input:[state], Qe:resQ})
            env.render()
            state = next_state
            print('\t', resQ, reward, state)
        print(done, reward)
            
            
        


0

	 [ 0.00194963] -2.3699013541227677e-06 [ -4.90624588e-01  -2.41402616e-04]
	 [ 0.00183259] -2.094231106898603e-06 [-0.49111976 -0.00049517]
	 [ 0.00172258] -1.8506027480566383e-06 [-0.49186459 -0.00074483]
	 [ 0.00161916] -1.6352930901910321e-06 [-0.49285313 -0.00098854]
	 [ 0.00152195] -1.4450138035617712e-06 [-0.49407764 -0.00122451]
	 [ 0.00143057] -1.2768560016440635e-06 [-0.49552863 -0.00145099]
	 [ 0.00134467] -1.1282505375279051e-06 [-0.49718486 -0.00165623]
	 [ 0.00126392] -9.969256007206237e-07 [-0.49904372 -0.00185886]
	 [ 0.00118802] -8.808724181092821e-07 [-0.50109103 -0.00204731]
	 [ 0.00111667] -7.783171420487103e-07 [-0.50331121 -0.00222017]
	 [ 0.00104959] -6.876904538307002e-07 [-0.50568738 -0.00237617]
	 [ 0.00098654] -6.076065007687046e-07 [-0.50820152 -0.00251414]
	 [ 0.00092727] -5.368399206008047e-07 [-0.51083457 -0.00263305]
	 [ 0.00087156] -4.7430736976107636e-07 [-0.5135666  -0.00273203]
	 [ 0.00081919] -4.190519699438268e-07 [-0.51637693 -0.00281033]
	 [ 

	 [  1.04784328e-06] -6.42368808043455e-13 [-0.49512249  0.00168033]
	 [  1.01797832e-06] -6.105474765433445e-13 [-0.49365548  0.00146701]
	 [  9.84798476e-07] -5.758253218703796e-13 [-0.49241275  0.00124273]
	 [  9.48302159e-07] -5.384732987806631e-13 [-0.49140357  0.00100918]
	 [  9.08514323e-07] -4.988605655252301e-13 [-0.49063549  0.00076808]
	 [  8.65508980e-07] -4.574342037608958e-13 [-0.49011423  0.00052126]
	 [  8.19388106e-07] -4.147037091023674e-13 [ -4.89843683e-01   2.70544643e-04]
	 [  7.70300119e-07] -3.712378604280869e-13 [ -4.89825872e-01   1.78110230e-05]
	 [  7.18421063e-07] -3.27642826985919e-13 [ -4.90060928e-01  -2.35055320e-04]
	 [  6.63957564e-07] -2.845526394032678e-13 [ -4.90547095e-01  -4.86167542e-04]
	 [  6.07152629e-07] -2.4260853577698907e-13 [-0.49128075 -0.00073365]
	 [  5.48272681e-07] -2.024439374396169e-13 [-0.49225641 -0.00097566]
	 [  4.87610066e-07] -1.6466362116176667e-13 [-0.49346679 -0.00121038]
	 [  4.25477737e-07] -1.2983656833405775e-13 [-0.4

	 [  6.14219061e-07] -1.993860431921653e-13 [-0.50334406  0.00241524]
	 [  6.28520695e-07] -2.1192340734659029e-13 [-0.50108065  0.00226342]
	 [  6.38108929e-07] -2.2158399036460174e-13 [-0.49898599  0.00209466]
	 [  6.42924135e-07] -2.2807628674594404e-13 [-0.49707577  0.00191023]
	 [  6.42919758e-07] -2.31213133855069e-13 [-0.49536425  0.00171152]
	 [  6.38100232e-07] -2.308935409446224e-13 [-0.49386424  0.00150001]
	 [  6.28503187e-07] -2.2713443125264481e-13 [-0.49258695  0.00127729]
	 [  6.14200133e-07] -2.2004224751482223e-13 [-0.49154191  0.00104504]
	 [  5.95297195e-07] -2.0983558784679287e-13 [-0.49073693  0.00080498]
	 [  5.71939893e-07] -1.9681360972135395e-13 [-0.49017802  0.00055891]
	 [  5.44301429e-07] -1.8137076119740463e-13 [ -4.89869349e-01   3.08673599e-04]
	 [  5.12588429e-07] -1.6396784696242985e-13 [ -4.89813217e-01   5.61324580e-05]
	 [  4.77038100e-07] -1.4512280610619075e-13 [ -4.90010044e-01  -1.96827405e-04]
	 [  4.37918146e-07] -1.253998388590352e-13 [ -4.90

	 [  5.07329332e-07] -1.259308194460034e-13 [-0.51415501  0.00284158]
	 [  5.39659368e-07] -1.4564479944793372e-13 [-0.51138425  0.00277076]
	 [  5.67943914e-07] -1.6446107915238347e-13 [-0.50870508  0.00267917]
	 [  5.91974981e-07] -1.8181979413350134e-13 [-0.50613758  0.0025675 ]
	 [  6.11563280e-07] -1.971996408476879e-13 [-0.50370098  0.0024366 ]
	 [  6.26570682e-07] -2.1014135149664016e-13 [-0.50141353  0.00228745]
	 [  6.36883613e-07] -2.2025816096109407e-13 [-0.49929234  0.00212118]
	 [  6.42426471e-07] -2.2724852188596362e-13 [-0.49735329  0.00193905]
	 [  6.43156966e-07] -2.309034914065158e-13 [-0.49561088  0.00174241]
	 [  6.39067935e-07] -2.3111246606766186e-13 [-0.49407813  0.00153275]
	 [  6.30194108e-07] -2.2787301430059775e-13 [-0.4927665   0.00131163]
	 [  6.16602676e-07] -2.2127761938153948e-13 [-0.49168579  0.00108072]
	 [  5.98392432e-07] -2.1152850837913155e-13 [-0.49084405  0.00084174]
	 [  5.75701563e-07] -1.989127273361212e-13 [-0.49024758  0.00059647]
	 [  5.487

	 [  4.65397960e-07] -1.0288658020419405e-13 [-0.51743209  0.00289664]
	 [  5.02083367e-07] -1.2289804179131759e-13 [-0.5145817   0.00285039]
	 [  5.35005654e-07] -1.4269546132862995e-13 [-0.51179893  0.00278277]
	 [  5.63918888e-07] -1.6168575931020361e-13 [-0.50910463  0.00269429]
	 [  5.88603257e-07] -1.7929935955855282e-13 [-0.50651901  0.00258562]
	 [  6.08876974e-07] -1.9500900338918427e-13 [-0.50406143  0.00245758]
	 [  6.24585823e-07] -2.0834581489664268e-13 [-0.50175031  0.00231113]
	 [  6.35614754e-07] -2.1891255841137446e-13 [-0.49960293  0.00214738]
	 [  6.41883730e-07] -2.2639051269659032e-13 [-0.49763536  0.00196756]
	 [  6.43343299e-07] -2.305608767693894e-13 [-0.49586233  0.00177303]
	 [  6.39984705e-07] -2.3129393975747456e-13 [-0.49429708  0.00156525]
	 [  6.31829778e-07] -2.2857212795470497e-13 [-0.49295132  0.00134577]
	 [  6.18946956e-07] -2.2247520786712705e-13 [-0.49183509  0.00111623]
	 [  6.01430827e-07] -2.1318679669018695e-13 [-0.49095673  0.00087836]
	 [  5.

...

	 [  3.54209618e-07] -3.848001572137774e-14 [-0.54408448  0.00511599]
	 [  4.52698487e-07] -7.377300483534344e-14 [-0.53881495  0.00526953]
	 [  5.47815660e-07] -1.1905443383894445e-13 [-0.53343134  0.00538361]
	 [  6.38843744e-07] -1.72976582980221e-13 [-0.527974    0.00545734]
	 [  7.25103178e-07] -2.339260955618886e-13 [-0.52248384  0.00549016]
	 [  8.05940658e-07] -3.000913335593643e-13 [-0.51700205  0.00548179]
	 [  8.80753532e-07] -3.6948615779733734e-13 [-0.51156973  0.00543232]
	 [  9.48978766e-07] -4.400460556822148e-13 [-0.50622762  0.00534212]
	 [  1.01010414e-06] -5.096553662923773e-13 [-0.50101573  0.00521189]
	 [  1.06366986e-06] -5.762356967411539e-13 [-0.49597309  0.00504264]
	 [  1.10927851e-06] -6.378036019882487e-13 [-0.49113741  0.00483568]
	 [  1.14659554e-06] -6.925131636320586e-13 [-0.4865448  0.0045926]
	 [  1.17533114e-06] -7.387341793001765e-13 [-0.48222954  0.00431527]
	 [  1.19528340e-06] -7.750916185167622e-13 [-0.47822375  0.00400579]
	 [  1.20630352e-06] 

	 [  3.15217790e-07] -2.770404853643272e-14 [-0.54609036  0.00504587]
	 [  4.14816924e-07] -5.881864863504809e-14 [-0.54087593  0.00521443]
	 [  5.11323833e-07] -1.0036636057469584e-13 [-0.53553199  0.00534395]
	 [  6.04022489e-07] -1.5110672043445454e-13 [-0.53009856  0.00543342]
	 [  6.92209994e-07] -2.0953317352470332e-13 [-0.5246164   0.00548217]
	 [  7.75224407e-07] -2.7389888931444853e-13 [-0.5191266  0.0054898]
	 [  8.52442724e-07] -3.422766653463566e-13 [-0.51367035  0.00545625]
	 [  9.23284404e-07] -4.126304506953305e-13 [-0.50828855  0.0053818 ]
	 [  9.87222734e-07] -4.828522389379387e-13 [-0.50302154  0.00526701]
	 [  1.04377591e-06] -5.508536903353501e-13 [-0.49790877  0.00511277]
	 [  1.09252051e-06] -6.145938945590995e-13 [-0.49298848  0.00492029]
	 [  1.13308840e-06] -6.721678056920317e-13 [-0.48829745  0.00469103]
	 [  1.16518265e-06] -7.218620108775498e-13 [-0.4838707   0.00442676]
	 [  1.18856758e-06] -7.621936531585813e-13 [-0.4797412  0.0041295]
	 [  1.20306572e-06]

	 [ -2.42328611e-07] -6.254482779191808e-14 [-0.57037169  0.00353362]
	 [ -1.39245785e-07] -3.0356194365754204e-14 [-0.56648842  0.00388327]
	 [ -3.51064813e-08] -9.421432415705078e-15 [-0.56228437  0.00420405]
	 [  6.93122217e-08] -3.6703306614809496e-16 [-0.55779083  0.00449354]
	 [  1.73227434e-07] -3.4697174783744404e-15 [-0.55304129  0.00474953]
	 [  2.75867933e-07] -1.8641889434940212e-14 [-0.54807123  0.00497007]
	 [  3.76462424e-07] -4.543852435297567e-14 [-0.54291779  0.00515344]
	 [  4.74255700e-07] -8.306383178598332e-14 [-0.53761953  0.00529825]
	 [  5.68515759e-07] -1.303976211971823e-13 [-0.53221616  0.00540338]
	 [  6.58529586e-07] -1.8603422113880711e-13 [-0.52674816  0.005468  ]
	 [  7.43624867e-07] -2.4830695442230843e-13 [-0.52125654  0.00549162]
	 [  8.23165749e-07] -3.153604946755396e-13 [-0.5157825   0.00547405]
	 [  8.96548158e-07] -3.8519495180014366e-13 [-0.51036707  0.00541543]
	 [  9.63229468e-07] -4.557147088285577e-13 [-0.50505086  0.00531621]
	 [  1.022700

	 [ -1.19897368e-06] -8.146881614994073e-13 [-0.57933239 -0.00283516]
	 [ -1.18137598e-06] -8.019599768874638e-13 [-0.58175149 -0.0024191 ]
	 [ -1.15496857e-06] -7.775040288224083e-13 [-0.58373665 -0.00198516]
	 [ -1.11994359e-06] -7.420503405070551e-13 [-0.58527322 -0.00153657]
	 [ -1.07657615e-06] -6.966608845681525e-13 [-0.58634986 -0.00107664]
	 [ -1.02518891e-06] -6.426952536398655e-13 [-0.58695863 -0.00060878]
	 [ -9.66162133e-07] -5.817558431047474e-13 [ -5.87095063e-01  -1.36430581e-04]
	 [ -8.99936424e-07] -5.156568764489829e-13 [ -5.86758143e-01   3.36920170e-04]
	 [ -8.27006659e-07] -4.4636609057040634e-13 [-0.58595035  0.00080779]
	 [ -7.47912736e-07] -3.7594765946539674e-13 [-0.58467765  0.00127271]
	 [ -6.63242531e-07] -3.064838215748558e-13 [-0.58294941  0.00172824]
	 [ -5.73624618e-07] -2.400483758337744e-13 [-0.58077838  0.00217103]
	 [ -4.79721166e-07] -1.7861704633824047e-13 [-0.57818061  0.00259777]
	 [ -3.82233537e-07] -1.2401368158336863e-13 [-0.57517529  0.003005

	 [ -1.19179083e-06] -7.683305923641357e-13 [-0.56817235 -0.00407157]
	 [ -1.20465620e-06] -7.960614322996138e-13 [-0.57191061 -0.00373826]
	 [ -1.20852258e-06] -8.122227114572108e-13 [-0.57528779 -0.00337718]
	 [ -1.20336392e-06] -8.163337839028018e-13 [-0.57827885 -0.00299106]
	 [ -1.18922662e-06] -8.082753422185093e-13 [-0.58086164 -0.00258279]
	 [ -1.16621868e-06] -7.883000298233303e-13 [-0.58301707 -0.00215543]
	 [ -1.13451301e-06] -7.570139187019244e-13 [-0.58472921 -0.00171214]
	 [ -1.09435132e-06] -7.153397347477262e-13 [-0.58598543 -0.00125622]
	 [ -1.04603316e-06] -6.645290461046621e-13 [-0.58677648 -0.00079105]
	 [ -9.89922455e-07] -6.06089820157349e-13 [ -5.87096524e-01  -3.20042590e-04]
	 [ -9.26436826e-07] -5.417706878721383e-13 [ -5.86943205e-01   1.53319005e-04]
	 [ -8.56047905e-07] -4.734811533426196e-13 [-0.58631765  0.00062555]
	 [ -7.79279333e-07] -4.032533003790161e-13 [-0.58522448  0.00109318]
	 [ -6.96707502e-07] -3.331737274788305e-13 [-0.58367174  0.00155274]
	